In [39]:
import datetime
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import ADASYN

from scipy.stats import loguniform, uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [40]:
logreg_distribution = {'penalty': ['l1', 'l2'], 'C': loguniform(0.001, 100), 'solver': ['newton-cg','sag','saga','lbfgs']}

Import data

In [41]:
X_train_3perc_lag3 = pd.read_excel("data/model_inputs2/x_train_3perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
X_test_3perc_lag3 = pd.read_excel("data/model_inputs2/x_test_3perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_train_3perc_lag3 = pd.read_excel("data/model_inputs2/y_train_3perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_test_3perc_lag3 = pd.read_excel("data/model_inputs2/y_test_3perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')

X_train_3perc_lag7 = pd.read_excel("data/model_inputs2/x_train_3perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
X_test_3perc_lag7 = pd.read_excel("data/model_inputs2/x_test_3perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_train_3perc_lag7 = pd.read_excel("data/model_inputs2/y_train_3perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_test_3perc_lag7 = pd.read_excel("data/model_inputs2/y_test_3perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')

X_train_5perc_lag3 = pd.read_excel("data/model_inputs2/x_train_5perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
X_test_5perc_lag3 = pd.read_excel("data/model_inputs2/x_test_5perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_train_5perc_lag3 = pd.read_excel("data/model_inputs2/y_train_5perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_test_5perc_lag3 = pd.read_excel("data/model_inputs2/y_test_5perc_lag3.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')

X_train_5perc_lag7 = pd.read_excel("data/model_inputs2/x_train_5perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
X_test_5perc_lag7 = pd.read_excel("data/model_inputs2/x_test_5perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_train_5perc_lag7 = pd.read_excel("data/model_inputs2/y_train_5perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_test_5perc_lag7 = pd.read_excel("data/model_inputs2/y_test_5perc_lag7.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')

Oversampling methods

In [42]:
def smote(X_train, y_train):
    sm = SMOTE(sampling_strategy='not majority')
    X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
    return X_train_oversampled, y_train_oversampled

def random_oversampler(X_train, y_train):
    oversample = RandomOverSampler(sampling_strategy='not majority')
    X_over, y_over = oversample.fit_resample(X_train, y_train)
    return X_over, y_over

def adasyn(X_train, y_train):
    ada = ADASYN(sampling_strategy = 'not majority')
    X_resampled, y_resampled = ADASYN().fit_resample(X_train, y_train)
    return X_resampled, y_resampled

12 combinations without tuning

In [43]:
perc_col, lag_col, oversampling_method, accuracy, precision, recall, f1, pred_count = [], [], [], [], [], [], [], []

for perc in ['3perc', '5perc']:
    for lag in [3, 7]:
        for oversampling in ['smote', 'random_oversampler', 'adasyn']:
            X_train = eval(f'X_train_{perc}_lag{lag}')
            X_test = eval(f'X_test_{perc}_lag{lag}')
            y_train = eval(f'y_train_{perc}_lag{lag}')
            y_test = eval(f'y_test_{perc}_lag{lag}')

            # oversampling
            if oversampling == 'smote':
                X_train_oversampled, y_train_oversampled = smote(X_train, y_train)
            elif oversampling == 'random_oversampler':
                X_train_oversampled, y_train_oversampled = random_oversampler(X_train, y_train)
            else:
                X_train_oversampled, y_train_oversampled = adasyn(X_train, y_train)

            # fit and predict
            logreg = LogisticRegression()
            pred = logreg.fit(X_train_oversampled, y_train_oversampled).predict(X_test)

            # update columns
            perc_col.append(perc)
            lag_col.append(lag)
            oversampling_method.append(oversampling)
            accuracy.append(accuracy_score(y_test, pred))
            precision.append(precision_score(y_test, pred, average='weighted'))
            recall.append(recall_score(y_test, pred, average='weighted'))
            f1.append(f1_score(y_test, pred, average='weighted'))
            pred_count.append(dict(pd.Series(pred).value_counts().sort_index()))

results_no_tuning = pd.DataFrame({
    'model': "Logistic Regression",
    'perc_threshold': perc_col,
    'lag': lag_col,
    'oversampling': oversampling_method,
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1,
    'pred_count': pred_count
})
results_no_tuning

,model,perc_threshold,lag,oversampling,accuracy,precision,recall,f1,pred_count
0,Logistic Regression,3perc,3,smote,0.452381,0.465545,0.452381,0.451517,"{-1: 64, 0: 74, 1: 30}"
1,Logistic Regression,3perc,3,random_oversampler,0.482143,0.493101,0.482143,0.479296,"{-1: 64, 0: 76, 1: 28}"
2,Logistic Regression,3perc,3,adasyn,0.458333,0.511533,0.458333,0.451369,"{-1: 83, 0: 48, 1: 37}"
3,Logistic Regression,3perc,7,smote,0.339286,0.401391,0.339286,0.333796,"{-1: 29, 0: 47, 1: 92}"
4,Logistic Regression,3perc,7,random_oversampler,0.363095,0.403882,0.363095,0.357793,"{-1: 32, 0: 54, 1: 82}"
5,Logistic Regression,3perc,7,adasyn,0.357143,0.394820,0.357143,0.350362,"{-1: 44, 0: 49, 1: 75}"
6,Logistic Regression,5perc,3,smote,0.738095,0.746075,0.738095,0.733575,"{-1: 9, 0: 143, 1: 16}"
7,Logistic Regression,5perc,3,random_oversampler,0.803571,0.749008,0.803571,0.765297,"{-1: 9, 0: 156, 1: 3}"
8,Logistic Regression,5perc,3,adasyn,0.708333,0.729507,0.708333,0.709673,"{-1: 8, 0: 140, 1: 20}"
9,Logistic Regression,5perc,7,smote,0.619048,0.663051,0.619048,0.637205,"{-1: 1, 0: 126, 1: 41}"


Tuning for best combination

In [44]:
perc_col, lag_col, oversampling_method, param, accuracy, precision, recall, f1 = [], [], [], [], [], [], [], []

perc = '5perc'
lag = 3
oversampling = 'random_oversampler'
X_train = eval(f'X_train_{perc}_lag{lag}')
X_test = eval(f'X_test_{perc}_lag{lag}')
y_train = eval(f'y_train_{perc}_lag{lag}')
y_test = eval(f'y_test_{perc}_lag{lag}')

# oversampling
if oversampling == 'smote':
    X_train_oversampled, y_train_oversampled = smote(X_train, y_train)
elif oversampling == 'random_oversampler':
    X_train_oversampled, y_train_oversampled = random_oversampler(X_train, y_train)
else:
    X_train_oversampled, y_train_oversampled = adasyn(X_train, y_train)

# tuning
logreg = LogisticRegression()
logreg_clf = RandomizedSearchCV(logreg, logreg_distribution, n_iter=100, scoring=['accuracy', 'recall_weighted', 'precision_weighted', 'f1_weighted'], refit='f1_weighted', random_state=42)
logreg_search = logreg_clf.fit(X_train_oversampled, y_train_oversampled)

# update columns
perc_col.append([perc]*100)
lag_col.append([lag]*100)
oversampling_method.append([oversampling]*100)
param.append(logreg_search.cv_results_['params'])
accuracy.append(logreg_search.cv_results_['mean_test_accuracy'])
precision.append(logreg_search.cv_results_['mean_test_precision_weighted'])
recall.append(logreg_search.cv_results_['mean_test_recall_weighted'])
f1.append(logreg_search.cv_results_['mean_test_f1_weighted'])

In [45]:
results = pd.DataFrame({
    'model': "Logistic Regression",
    'perc_threshold': np.array(perc_col).flatten(),
    'lag': np.array(lag_col).flatten(),
    'oversampling': np.array(oversampling_method).flatten(),
    'parameters': np.array(param).flatten(),
    'accuracy': np.array(accuracy).flatten(),
    'precision': np.array(precision).flatten(),
    'recall': np.array(recall).flatten(),
    'f1': np.array(f1).flatten(),
}).sort_values(by="f1", ascending=False).reset_index(drop=True)

In [46]:
results.head(10)

,model,perc_threshold,lag,oversampling,parameters,accuracy,precision,recall,f1
0,Logistic Regression,5perc,3,random_oversampler,"{'C': 0.006026889128682509, 'penalty': 'l1', '...",0.700302,0.719577,0.700302,0.698334
1,Logistic Regression,5perc,3,random_oversampler,"{'C': 0.001339971723117972, 'penalty': 'l1', '...",0.695888,0.718191,0.695888,0.694819
2,Logistic Regression,5perc,3,random_oversampler,"{'C': 0.0010656401760606444, 'penalty': 'l1', ...",0.691479,0.712268,0.691479,0.690184
3,Logistic Regression,5perc,3,random_oversampler,"{'C': 0.0012674255898937226, 'penalty': 'l2', ...",0.662685,0.664080,0.662685,0.653345
4,Logistic Regression,5perc,3,random_oversampler,"{'C': 0.0010833297073737746, 'penalty': 'l2', ...",0.659900,0.663952,0.659900,0.651294
5,Logistic Regression,5perc,3,random_oversampler,"{'C': 0.0015991003111214825, 'penalty': 'l2', ...",0.661296,0.660391,0.661296,0.650203
6,Logistic Regression,5perc,3,random_oversampler,"{'C': 0.0014857392806279237, 'penalty': 'l2', ...",0.659436,0.659643,0.659436,0.649256
7,Logistic Regression,5perc,3,random_oversampler,"{'C': 0.0016832027985721903, 'penalty': 'l2', ...",0.660135,0.658807,0.660135,0.648767
8,Logistic Regression,5perc,3,random_oversampler,"{'C': 0.0019517224641449492, 'penalty': 'l2', ...",0.660136,0.657225,0.660136,0.647734
9,Logistic Regression,5perc,3,random_oversampler,"{'C': 0.0021147447960615704, 'penalty': 'l2', ...",0.658976,0.655355,0.658976,0.646270


Best Model

In [47]:
best_params, best_perc, best_lag, best_oversampling = results.iloc[0]['parameters'], results.iloc[0]['perc_threshold'], results.iloc[0]['lag'], results.iloc[0]['oversampling']
best_model = LogisticRegression(**best_params)
X_train, y_train, x_test, y_test = eval(f'X_train_{best_perc}_lag{best_lag}'), eval(f'y_train_{best_perc}_lag{best_lag}'), eval(f'X_test_{best_perc}_lag{best_lag}'), eval(f'y_test_{best_perc}_lag{best_lag}')

if best_oversampling == 'smote':
    X_train_oversampled, y_train_oversampled = smote(X_train, y_train)
elif best_oversampling == 'random_oversampler':
    X_train_oversampled, y_train_oversampled = random_oversampler(X_train, y_train)
else:
    X_train_oversampled, y_train_oversampled = adasyn(X_train, y_train)

best_model.fit(X_train_oversampled, y_train_oversampled)
best_model

LogisticRegression(C=0.006026889128682509, penalty='l1', solver='saga')

In [48]:
def print_results(actual, predictions):
    print(f"accuracy: {accuracy_score(actual, predictions)}")
    print(f"precision: {precision_score(actual, predictions, average='weighted')}")
    print(f"recall: {recall_score(actual, predictions, average='weighted')}")
    print(f"f1: {f1_score(actual, predictions, average='weighted')}")
    print(f"confusion matrix:\n{confusion_matrix(actual, predictions)}")

Predict on test set

In [49]:
test_pred = best_model.predict(X_test)
print_results(y_test, test_pred)

accuracy: 0.75
precision: 0.7459415584415584
recall: 0.75
f1: 0.7438158593509797
confusion matrix:
[[ 12   9   0]
 [ 23 114   0]
 [  1   9   0]]


Predict on full data

In [50]:
X_full = pd.concat([X_train, X_test])
y_full = pd.concat([y_train, y_test])

full_pred = best_model.predict(X_full)
print_results(y_full, full_pred)

accuracy: 0.7665276950565812
precision: 0.9217839401019193
recall: 0.7665276950565812
f1: 0.8252744374095169
confusion matrix:
[[  48   16    2]
 [ 183 1221  169]
 [   4   18   18]]


Business Metric

In [51]:
whole_df_x = pd.concat([X_train, X_test])
whole_df_x.shape

(1679, 11)

In [52]:
whole_df_x['year'] = whole_df_x.index.year

In [53]:
business_metric_results = pd.DataFrame(columns=['exp_strategy_annual_return', 'exp_benchmark_annual_return'])

for year in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    year_data = whole_df_x[whole_df_x['year'] == year]
    # year_data = year_data.set_index('index')
    year_data = year_data.drop(['year'], axis = 1)
    predict_x = best_model.predict(np.array(year_data)) 
    # predictions = np.argmax(predict_x,axis=1)
    predictions = predict_x

    df_pred = pd.DataFrame({'prediction':predictions}, index=year_data.index)
    df_pred = df_pred.replace({-1:1, 1:-1}) # convert classes to buy hold sell
    dates = df_pred.index

    if year == 2022:
        end_date = "2022-09-02"
    else:
        end_date = str(year+1) + "-01-01"
    df_prices = yf.download("^GSPC", start=dates[0], end=end_date)[['Adj Close']]

    # create positions column
    positions = []
    prev = 0
    for i in range(len(df_pred)):
        if df_pred.iloc[i]['prediction'] == 0:
            positions.append(prev)
        else:
            prev = df_pred.iloc[i]['prediction']
            positions.append(prev)

    df_business = pd.DataFrame()
    df_business['stock_daily_log_return'] = np.log(df_prices /df_prices.shift(1))['Adj Close']
    df_business['prediction'] = df_pred['prediction']
    df_business['position'] = positions
    df_business['benchmark'] = 1 # long and hold strategy
    df_business["strategy_Returns"] = df_business["stock_daily_log_return"] * df_business["position"].shift(1)
    df_business["benchmark_Returns"] = df_business["stock_daily_log_return"] * df_business["benchmark"].shift(1)

    # Annual Mean Returns or Expected returns
    expected_strategy_annual_return = np.exp(df_business['strategy_Returns'].mean() * 252) - 1 
    expected_benchmark_annual_return = np.exp(df_business['benchmark_Returns'].mean() * 252) - 1 
    business_metric_results.loc[year] = [expected_strategy_annual_return, expected_benchmark_annual_return]
    # print(f'Expected Annual Returns: Strategy: {round(expected_strategy_annual_return*100, 2)}%  |  Stock: {round(expected_benchmark_annual_return*100, 2)}%')

business_metric_results

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,exp_strategy_annual_return,exp_benchmark_annual_return
2016,0.164536,0.112846
2017,0.173690,0.185753
2018,-0.113091,-0.070634
2019,0.288443,0.288443
2020,-0.504476,0.152929
2021,0.284494,0.289230
2022,-0.218343,-0.249185
